In [4]:
# IMPORTING LIBRARIES REQUIRED FOR THIS CODE
import sys
import numpy as np
from scipy.stats import randint
import scipy.io as sio
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns
from sklearn.model_selection import train_test_split
# from sklearn.cross_validation import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import pipeline
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

In [5]:
#from keras.processing import sequence
from tensorflow.keras.models import load_model
import tensorflow.keras.backend as K
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
#from keras.layers import embedding
from tensorflow.keras.models  import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD 
from tensorflow.keras.callbacks import EarlyStopping
#from keras.utils import np_utils
import itertools
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D

In [6]:
## loading autospectrum file
## This file is of size 500 x 23 x 249
## 23 is the number of subsets for which autospectrum is calculated for every dataset containing 2050 datapoints.
## That is, 22 subsets containing 500 to 1025 datapoints at an increment of 25, 
## and finally the whole dataset of 2050 datapoints as 23rd.
## autospectrum is caculated for such 249 datasets containing 2050 datapoints.
## autospectrum is calculated for 500 frequnecy points.

indata1 = sio.loadmat('logmldata500.mat')['pwwseq']

### input data contains the 22 subsets, while the output data contains the autospectrum for 2050 datapoints
indata2 = np.swapaxes(indata1,0,2)
indata = np.swapaxes(indata2,0,1)
dfin, dfout = indata[:22][:][:], indata[22:][:][:]
dfin = np.swapaxes(dfin, 0,1)
dfout = np.swapaxes(dfout, 0,1)
dfin.shape

## shuffling the datasets to randomize datasets for splits for corss-validation
ind = np.random.RandomState(4).permutation(range(len(dfin)))
dfin, dfout = dfin[ind][:][:], dfout[ind][:][:]


In [4]:
nk = 1
n = str(nk)

In [5]:
## cross validation. 

for train_index, test_index in KFold(n_splits=5).split(dfin):
    train_X, test_X = dfin[train_index][:][:], dfin[test_index][:][:]
    train_y, test_y = dfout[train_index][:][:], dfout[test_index][:][:]
    train_X = np.swapaxes(train_X, 0,1)
    train_y = np.swapaxes(train_y,  0,1)
    test_X = np.swapaxes(test_X, 0,1)
    test_y = np.swapaxes(test_y,  0,1)
    
    train_X = np.reshape(train_X,(train_X.shape[0],train_X.shape[1]*train_X.shape[2]))
    train_y = np.reshape(train_y,(train_y.shape[0],train_y.shape[1]*train_y.shape[2]))
    test_X = np.reshape(test_X,(test_X.shape[0],test_X.shape[1]*test_X.shape[2]))
    test_y = np.reshape(test_y,(test_y.shape[0],test_y.shape[1]*test_y.shape[2]))
    train_X = np.swapaxes(train_X, 0,1)
    train_y = np.swapaxes(train_y,  0,1)
    test_X = np.swapaxes(test_X, 0,1)
    test_y = np.swapaxes(test_y,  0,1)

    train_X = np.reshape(train_X,(train_X.shape[0],train_X.shape[1],1))
    test_X = np.reshape(test_X,(test_X.shape[0],test_X.shape[1],1))

    #train_X = np.reshape(train_X,(train_X.shape[0],train_X.shape[1],1))
    #test_X = np.reshape(test_X,(test_X.shape[0],test_X.shape[1],1))
    #train_y = np.reshape(train_y,(train_y.shape[0],train_y.shape[1],1))
    #test_y = np.reshape(test_y,(test_y.shape[0],test_y.shape[1],1))





    #print(indata[4][15][0],indata1[0][4][15],train_X2[4][5][492],train_X[2997][4],ind[5])
    print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
    
    #best_stop = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 100)
    
    best_stop = [
    tf.keras.callbacks.EarlyStopping(patience=100, monitor = 'val_loss', verbose = 1),
    tf.keras.callbacks.ModelCheckpoint(filepath='models/filt_best_apt_model' + n + '.h5', save_best_only = True),
    tf.keras.callbacks.TensorBoard(log_dir='./logs')
    ]
    
    
    model = Sequential()
    model.add(LSTM(300,input_shape=(train_X.shape[1],train_X.shape[2])))
    #add(Activation('sigmoid'))
    model.add(Dropout(0.5))



    #model.add(Flatten())
    #model.add(Dense(5))
    model.add(Dense(1))
    #model.add(Activation('sigmoid'))
    epochs = 2000
    batch_size = 501
    model.compile(loss='mean_squared_error', optimizer = 'adam')
    print(model.summary())
    pww = model.fit(train_X, train_y, epochs=epochs, batch_size=batch_size, validation_data = (test_X,test_y), shuffle = 'False', callbacks = best_stop)
    model.load_weights('models/filt_best_apt_model'+n+'.h5')
    yhat = model .predict(test_X)
    testset = ind[test_index]
    sio.savemat('Saved_outfiles/4-kf-filtlogpww'+n+'.mat',{'yhat':yhat, 'test_y' :test_y, 'testset' :testset})
    nk = nk+1
    n = str(nk)

(32000, 22, 1) (32000, 1) (8500, 22, 1) (8500, 1)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 300)               362400    
_________________________________________________________________
dropout (Dropout)            (None, 300)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 301       
Total params: 362,701
Trainable params: 362,701
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/2000


KeyboardInterrupt: 

In [32]:
len(pww.history['val_loss'])
test_index

array([100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
       113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123])

In [30]:
yhat = model.predict(test_X)
#test_X = test_X.reshape((test_X.shape[0], 1000))
#yhat = yhat.reshape((test_X.shape[0], 4))
#inv_yhat = np.concatenate((yhat,test_X), axis=1)
#scaler = StandardScaler()
#scaler.fit(inv_yhat)
#inv_yhat = scaler.inverse_transform(inv_yhat)
#inv_yhat = inv_yhat[:,0]
# invert scaling for actual
#test_y = test_y.reshape((test_y.shape[0], 1))
#inv_y = np.concatenate((test_y, test_X), axis=1)
#inv_y = scaler.inverse_transform(inv_y)
#inv_y = inv_y[:,0]
# calculate RMSE
#rmse = np.sqrt(mean_squared_error(inv_y, inv_yhat))
#print('Test RMSE: %.3f' % rmse)

In [8]:
sio.savemat('4-24-501-300logpww_filt_mse.mat',{'yhat':yhat, 'test_y' :test_y, 'testset' :testset})
# naming is set at out~shuffle-testsize-batchsize-epoch

In [29]:
indata1[4][22][113]

413.34957330563617

In [32]:
ythat = model.predict(train_X)
trainset = ind[:n_train_time]

In [33]:
sio.savemat('2-49-501-300train_mse.mat',{'ythat':ythat, 'train_y' :train_y, 'trainset' :trainset})